In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

# MakeDatasets

In [3]:
!pip install filterpy
import random
import pandas as pd
import numpy as np
import os
import glob
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
from tqdm import tqdm
from scipy.signal import butter, lfilter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 9.9 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=b80114fdac3f2c75c7684d98ee5e86a8186a604f69a58453c46a91154470be61
  Stored in directory: /root/.cache/pip/wheels/fe/f6/cb/40331472edf4fd399b8cad02973c6acbdf26898342928327fe
Successfully built filterpy


In [4]:
def LPF(series, low, order=1):
    '''
    series : 데이터
    low : 최저 구간(0< low < 1)
    order : 필터 계수, 높을수록 민감
    '''
    
    b, a = butter(
                  N = order,
                  Wn = low,
                  btype = 'low',
                  )
    lpf_series = lfilter(b, a, series)
    
    return lpf_series

In [10]:
'''
def preprocessing_2(input_path):
    #all_input_list = sorted(glob.glob(input_path))
    for datapath in input_path:
        #data = pd.read_csv(datapath)  # 이게 지금 28일치 * 24시
        data['obs_time'] = data.index % 24  # 시간통일
        col_list = data.columns
        for i in range(0, len(col_list)):
            col = col_list[i]
            if '누적' in col:
                data[col] = data.groupby((data.obs_time == 0).cumsum()).agg(
                    col_list[i-1]).cumsum()
    return data'''

In [5]:
def preprocessing_2(data):
  data['obs_time'] = data.index % 24  # 시간통일
  col_list = data.columns
  for i in range(0, len(col_list)):
      col = col_list[i]
      if '누적' in col:
          data[col] = data.groupby((data.obs_time == 0).cumsum()).agg(
              col_list[i-1]).cumsum()
  return data

In [25]:

def makedatasets(input_path):
  print("###################################################################################################################")


  test = preprocessing_2(input_path) #input_path : 생성될 데이터셋 path
  print(test)
  test['05시내부온도관측치누적'] = 0
  test['19시내부온도관측치누적'] = 0
  test['23시내부온도관측치누적'] = 0
  print("###################################################################################################################")

  test['05시내부습도관측치누적'] = 0
  test['19시내부습도관측치누적'] = 0
  test['23시내부습도관측치누적'] = 0

  test['05시co2관측치누적'] = 0
  test['19시co2관측치누적'] = 0
  test['23시co2관측치누적'] = 0

  test['05시ec관측치누적'] = 0
  test['19시ec관측치누적'] = 0
  test['23시ec관측치누적'] = 0

  test['05시분무량누적'] = 0
  test['19시분무량누적'] = 0
  test['23시분무량누적'] = 0

  test['05시백색광누적'] = 0
  test['19시백색광누적'] = 0
  test['23시백색광누적'] = 0

  test['05시적색광누적'] = 0
  test['19시적색광누적'] = 0
  test['23시적색광누적'] = 0

  test['05시청색광누적'] = 0
  test['19시청색광누적'] = 0
  test['23시청색광누적'] = 0

  test['05시총광량누적'] = 0
  test['19시총광량누적'] = 0
  test['23시총광량누적'] = 0
  print("#############################################zxczxczxczxczxc####################################################")

  test = test.drop(test.filter(regex='일간누적').columns, axis=1)

  test_new = test
  print("#############################################qqqqqqqqqqqqqqqqqq#######################################################")

  test['시간대'] = 0
  test['시간대'][(test['obs_time'] >= 0) & (test['obs_time'] <= 5)] = 1
  test['시간대'][(test['obs_time'] > 5) & (test['obs_time'] < 20)] = 2
  test['시간대'][(test['obs_time'] >= 20) & (test['obs_time'] <= 23)] = 3

  #########################################################교체#####################

  print("###################################################################################################################")
  test['측정될수분량2'] = 0
  print(test.shape)
  for i in range(22,len(test),24):
    s = test.iloc[i,6] + test.iloc[i+1,6] * ((test.iloc[i, 3] + test.iloc[i+1,3]) / 2)
    test.iloc[i, 40] = s

  print("########################################################aaaa###########################################################")


  test['측정될수분량1'] = 0
  for i in range(22,len(test),24):
    s = test.iloc[i+1,6] * (test.iloc[i+1,3])
    test.iloc[i, 41] = s



  print("########################################################sssss###########################################################")


  test['측정될수분량3'] = 0
  for i in range(22,len(test),24):
    s = (test.iloc[i,6] + test.iloc[i+1,6]) * ((test.iloc[i, 3] + test.iloc[i+1,3]) / 2)
    test.iloc[i, 42] = s

  ###################################################################################################################

  print("########################################################22222222###########################################################")
  test_x = test
  test_x = test_x.groupby(['DAT','Case','시간대']).sum().reset_index()

  print("####################################################433333334###############################################################")

  test_x = test_x.sort_values(by=['Case','DAT','시간대'], axis=0).reset_index()
  test_x.drop(['index'], axis = 1, inplace=True)


  t = test_x.groupby(['Case'])['내부온도관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t.drop(['level_1'], axis = 1, inplace = True)
  test_x['내부온도관측치'] = t

  t2 = test_x.groupby(['Case'])['내부습도관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t2.drop(['level_1'], axis = 1, inplace = True)
  test_x['내부습도관측치'] = t2

  t3 = test_x.groupby(['Case'])['co2관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t3.drop(['level_1'], axis = 1, inplace = True)
  test_x['co2관측치'] = t3

  t4 = test_x.groupby(['Case'])['ec관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t4.drop(['level_1'], axis = 1, inplace = True)
  test_x['ec관측치'] = t4

  t5 = test_x.groupby(['Case'])['시간당분무량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t5.drop(['level_1'], axis = 1, inplace = True)
  test_x['시간당분무량'] = t5

  t6 = test_x.groupby(['Case'])['시간당백색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t6.drop(['level_1'], axis = 1, inplace = True)
  test_x['시간당백색광량'] = t6

  t7 = test_x.groupby(['Case'])['시간당적색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t7.drop(['level_1'], axis = 1, inplace = True)
  test_x['시간당적색광량'] = t7

  t8 = test_x.groupby(['Case'])['시간당청색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t8.drop(['level_1'], axis = 1, inplace = True)
  test_x['시간당청색광량'] = t8

  t9 = test_x.groupby(['Case'])['시간당총광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t9.drop(['level_1'], axis = 1, inplace = True)
  test_x['시간당총광량'] = t9

  test_new_x = test_new
  test_new_x = test_new_x.groupby(['DAT','Case']).sum().reset_index()

  test_new_x = test_new_x.sort_values(by=['Case','DAT'], axis=0).reset_index()
  test_new_x.drop(['index'], axis = 1, inplace=True)

  print("####################################################44444###############################################################")

  for i in range(len(test_new_x)):
    #온도
    test_new_x.iloc[i,12] = test_x.iloc[0 + (3*i),4]
    test_new_x.iloc[i,13] = test_x.iloc[1 + (3*i),4]
    test_new_x.iloc[i,14] = test_x.iloc[2 + (3*i),4]
    #습도
    test_new_x.iloc[i,15] = test_x.iloc[0 + (3*i),5]
    test_new_x.iloc[i,16] = test_x.iloc[1 + (3*i),5]
    test_new_x.iloc[i,17] = test_x.iloc[2 + (3*i),5]
    #co2
    test_new_x.iloc[i,18] = test_x.iloc[0 + (3*i),6]
    test_new_x.iloc[i,19] = test_x.iloc[1 + (3*i),6]
    test_new_x.iloc[i,20] = test_x.iloc[2 + (3*i),6]
    #ec
    test_new_x.iloc[i,21] = test_x.iloc[0 + (3*i),7]
    test_new_x.iloc[i,22] = test_x.iloc[1 + (3*i),7]
    test_new_x.iloc[i,23] = test_x.iloc[2 + (3*i),7]
    #분무
    test_new_x.iloc[i,24] = test_x.iloc[0 + (3*i),8]
    test_new_x.iloc[i,25] = test_x.iloc[1 + (3*i),8]
    test_new_x.iloc[i,26] = test_x.iloc[2 + (3*i),8]
    #백색
    test_new_x.iloc[i,27] = test_x.iloc[0 + (3*i),9]
    test_new_x.iloc[i,28] = test_x.iloc[1 + (3*i),9]
    test_new_x.iloc[i,29] = test_x.iloc[2 + (3*i),9]
    #적색
    test_new_x.iloc[i,30] = test_x.iloc[0 + (3*i),10]
    test_new_x.iloc[i,31] = test_x.iloc[1 + (3*i),10]
    test_new_x.iloc[i,32] = test_x.iloc[2 + (3*i),10]
    #청색
    test_new_x.iloc[i,33] = test_x.iloc[0 + (3*i),11]
    test_new_x.iloc[i,34] = test_x.iloc[1 + (3*i),11]
    test_new_x.iloc[i,35] = test_x.iloc[2 + (3*i),11]
    #총광
    test_new_x.iloc[i,36] = test_x.iloc[0 + (3*i),12]
    test_new_x.iloc[i,37] = test_x.iloc[1 + (3*i),12]
    test_new_x.iloc[i,38] = test_x.iloc[2 + (3*i),12]

  print("#####################################################555555##############################################################")


  test_new_x.drop(['obs_time','시간대','내부온도관측치','내부습도관측치','co2관측치','ec관측치'], axis = 1, inplace = True)

  test_x = test_new_x

  test_x = test_x.drop(test_x.filter(regex='시간당').columns, axis=1)


  test_x['하루평균온도'] = (test_x['05시내부온도관측치누적'] + test_x['19시내부온도관측치누적'] + test_x['23시내부온도관측치누적']) / 3
  test_x['하루평균습도'] = (test_x['05시내부습도관측치누적'] + test_x['19시내부습도관측치누적'] + test_x['23시내부습도관측치누적']) / 3
  test_x['하루평균co2'] = (test_x['05시co2관측치누적'] + test_x['19시co2관측치누적'] + test_x['23시co2관측치누적']) / 3
  test_x['하루평균ec'] = (test_x['05시ec관측치누적'] + test_x['19시ec관측치누적'] + test_x['23시ec관측치누적']) / 3
  test_x['하루평균분무량'] = (test_x['05시분무량누적'] + test_x['19시분무량누적'] + test_x['23시분무량누적']) / 3
  test_x['하루평균백색광'] = (test_x['05시백색광누적'] + test_x['19시백색광누적'] + test_x['23시백색광누적']) / 3
  test_x['하루평균적색광'] = (test_x['05시적색광누적'] + test_x['19시적색광누적'] + test_x['23시적색광누적']) / 3
  test_x['하루평균청색광'] = (test_x['05시청색광누적'] + test_x['19시청색광누적'] + test_x['23시청색광누적']) / 3
  test_x['하루평균총광량'] = (test_x['05시총광량누적'] + test_x['19시총광량누적'] + test_x['23시총광량누적']) / 3

  test_x = test_x.drop(test_x.filter(regex = '총광').columns, axis =1)


  test_x['ec_x_분무05'] = (test_x['05시ec관측치누적']+1) * (test_x['05시분무량누적']+1)
  test_x['ec_x_분무19'] = (test_x['19시ec관측치누적']+1) * (test_x['19시분무량누적']+1)
  test_x['ec_x_분무23'] = (test_x['23시ec관측치누적']+1) * (test_x['23시분무량누적']+1)
  test_x['ec_x_분무평균'] = (test_x['하루평균ec']+1) * (test_x['하루평균분무량']+1)




  test_x['적색_+_청색05'] = (test_x['05시적색광누적']) + (test_x['05시청색광누적'])
  test_x['적색_+_청색19'] = (test_x['19시적색광누적']) + (test_x['19시청색광누적'])
  test_x['적색_+_청색23'] = (test_x['23시적색광누적']) + (test_x['23시청색광누적'])
  test_x['적색_+_청색평균'] = (test_x['하루평균적색광']) + (test_x['하루평균청색광'])


  test_x['수분량합'] =  test_x['측정될수분량1'] + test_x['측정될수분량2'] + test_x['측정될수분량3']

  test_x['수분량합12'] =  test_x['측정될수분량1'] + test_x['측정될수분량2']

  test_x['수분량합13'] =  test_x['측정될수분량1'] + test_x['측정될수분량3']

  test_x['수분량합23'] =  test_x['측정될수분량2'] + test_x['측정될수분량3']

  test_x.drop(test_x.filter(regex = '백색'), axis = 1, inplace=True)

  test_x.drop(test_x.filter(regex = '청색'), axis = 1, inplace=True)

  test_x.drop(['측정될수분량2','측정될수분량1','측정될수분량3'],axis=1,inplace=True)

  basic_col = test_x.columns
  kal_test = test_x

  for i in range(2,34):
    test_x['kf_X_'+str(i)]=0

  for j in range(0,len(test_x.Case.unique())): #수정 Data_Leakage
    kal = test_x[test_x['Case'] == j]
    for i in tqdm(range(len(basic_col))):
        if((i == 0) | (i == 1)):
          continue
        current=0
        sum_c=[]
        z = kal.loc[:, kal.columns[i]]
        a = []           #필터링 된 피쳐(after)
        b = []           #필터링 전 피쳐(before)
        my_filter = KalmanFilter(dim_x=2,dim_z=1) #create kalman filter
        my_filter.x = np.array([[2.],[0.]])       # initial state (location and velocity)
        my_filter.F = np.array([[1.,1.], [0.,1.]])    # state transition matrix
        my_filter.H = np.array([[1.,0.]])    # Measurement function
        my_filter.P *= 1000.                 # covariance matrix
        my_filter.R = 5                      # state uncertainty
        my_filter.Q = Q_discrete_white_noise(dim = 2,dt=.1,var=.5) # process uncertainty   
        for k in z.values:
            my_filter.predict()
            my_filter.update(k)
            # do something with the output
            x = my_filter.x
            a.extend(x[0])
            b.append(k)
        sum_c=sum_c+a
        test_x['kf_X_'+str(i)][test_x['Case'] == j] = sum_c

  #fillna도 해줘야함

  copy_test_x = test_x.copy()

  copy_test_x.drop(copy_test_x.filter(regex = 'kf').columns, axis = 1, inplace = True)
  print(copy_test_x)

  raw_cols_te = copy_test_x.columns
  new_test_x = pd.DataFrame()
  for i in range(0,len(copy_test_x.Case.unique())):
    target = copy_test_x[copy_test_x['Case'] == i]
    mean_arr = []
    median_arr = []


    pbar = (target.columns[:])
    if(i == 1):
      print('1')
      print(pbar)
    if(i == 2):
      print('2')
      print(pbar)
    for column in pbar:
        column_list = target[column].to_list()
        for i in range(7):
          if((i) == (len(column_list))):
            break
          mean_arr.append(column_list[i])
          median_arr.append(column_list[i])

            
        for i in range(7, len(column_list)):
            mean_arr.append(float(np.mean(column_list[i-7:i])))
            median_arr.append(float(np.median(column_list[i-7:i])))
        target[f'{column}_mean_7'] = mean_arr
        target[f'{column}_median_7'] = median_arr
        mean_arr = []
        median_arr = []
    
    new_test_x = pd.concat([new_test_x, target], axis=0)
  test_x_moving_7 = new_test_x.drop(raw_cols_te, axis=1)

  test_x = pd.concat([test_x, test_x_moving_7], axis=1)

  raw_cols_te = copy_test_x.columns



  new_test_x = pd.DataFrame()

  for i in range(0,len(copy_test_x.Case.unique())):
    target = copy_test_x[copy_test_x['Case'] == i]
    mean_arr = []
    median_arr = []


    pbar = tqdm(target.columns[:])
    for column in pbar:
        column_list = target[column].to_list()
        for i in range(14):
          if((i) == (len(column_list))):
            break
          mean_arr.append(column_list[i])
          median_arr.append(column_list[i])
            
        for i in range(14, len(column_list)):
            mean_arr.append(float(np.mean(column_list[i-14:i])))
            median_arr.append(float(np.median(column_list[i-14:i])))
        target[f'{column}_mean_14'] = mean_arr
        target[f'{column}_median_14'] = median_arr
        mean_arr = []
        median_arr = []
    
    new_test_x = pd.concat([new_test_x, target], axis=0)
  test_x_moving_14 = new_test_x.drop(raw_cols_te, axis=1)

  test_x = pd.concat([test_x, test_x_moving_14], axis=1)

  test_x_fil = test_x.iloc[:,1:34]


  new_test_x = pd.DataFrame()
  for i in range(0,len(copy_test_x.Case.unique())):
    target = test_x_fil[test_x_fil['Case'] == i]
    test_merge = LPF(target,0.1,1)
    test_merge = pd.DataFrame(test_merge)
    
    new_test_x = pd.concat([new_test_x, test_merge], axis=0)
  new_test_x = new_test_x.add_suffix('_LPF')
  new_test_x = new_test_x.reset_index(drop = True)
  test_x = pd.concat([test_x,new_test_x], axis = 1)


  test_x.drop(['0_LPF'],axis = 1, inplace=True)
  test_x.drop(test_x.filter(regex = 'Case'),axis=1, inplace=True)

  return test_x

# 2일부터

In [30]:
def cumsum_dataset(df1, df2):
    newdf = pd.concat([df1, df2], axis = 0)
    return newdf

In [39]:

def make_data(input_path, fix_path, day):
  test = preprocessing_2(input_path) 
  #ctgan = input_path.copy()
  #ctgan = test

  if day != 0 : 
      test = pd.concat([test,fix_path],axis=0)
  test = test.reset_index(drop = True)
  #test = ctgan


  test['05시내부온도관측치누적'] = 0
  test['19시내부온도관측치누적'] = 0
  test['23시내부온도관측치누적'] = 0

  test['05시내부습도관측치누적'] = 0
  test['19시내부습도관측치누적'] = 0
  test['23시내부습도관측치누적'] = 0

  test['05시co2관측치누적'] = 0
  test['19시co2관측치누적'] = 0
  test['23시co2관측치누적'] = 0

  test['05시ec관측치누적'] = 0
  test['19시ec관측치누적'] = 0
  test['23시ec관측치누적'] = 0

  test['05시분무량누적'] = 0
  test['19시분무량누적'] = 0
  test['23시분무량누적'] = 0

  test['05시백색광누적'] = 0
  test['19시백색광누적'] = 0
  test['23시백색광누적'] = 0

  test['05시적색광누적'] = 0
  test['19시적색광누적'] = 0
  test['23시적색광누적'] = 0

  test['05시청색광누적'] = 0
  test['19시청색광누적'] = 0
  test['23시청색광누적'] = 0

  test['05시총광량누적'] = 0
  test['19시총광량누적'] = 0
  test['23시총광량누적'] = 0

  test = test.drop(test.filter(regex='일간누적').columns, axis=1)

  test_new = test

  test['시간대'] = 0
  test['시간대'][(test['obs_time'] >= 0) & (test['obs_time'] <= 5)] = 1
  test['시간대'][(test['obs_time'] > 5) & (test['obs_time'] < 20)] = 2
  test['시간대'][(test['obs_time'] >= 20) & (test['obs_time'] <= 23)] = 3

  #########################################################교체#####################

  test['측정될수분량2'] = 0
  for i in range(22,len(test),24):
    s = test.iloc[i,6] + test.iloc[i+1,6] * ((test.iloc[i, 3] + test.iloc[i+1,3]) / 2)
    test.iloc[i, 40] = s



  test['측정될수분량1'] = 0
  for i in range(22,len(test),24):
    s = test.iloc[i+1,6] * (test.iloc[i+1,3])
    test.iloc[i, 41] = s





  test['측정될수분량3'] = 0
  for i in range(22,len(test),24):
    s = (test.iloc[i,6] + test.iloc[i+1,6]) * ((test.iloc[i, 3] + test.iloc[i+1,3]) / 2)
    test.iloc[i, 42] = s

  ###################################################################################################################

  test_x = test
  test_x = test_x.groupby(['DAT','Case','시간대']).sum().reset_index()


  test_x = test_x.sort_values(by=['Case','DAT','시간대'], axis=0).reset_index()
  test_x.drop(['index'], axis = 1, inplace=True)


  t = test_x.groupby(['Case'])['내부온도관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t.drop(['level_1'], axis = 1, inplace = True)
  test_x['내부온도관측치'] = t

  t2 = test_x.groupby(['Case'])['내부습도관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t2.drop(['level_1'], axis = 1, inplace = True)
  test_x['내부습도관측치'] = t2

  t3 = test_x.groupby(['Case'])['co2관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t3.drop(['level_1'], axis = 1, inplace = True)
  test_x['co2관측치'] = t3

  t4 = test_x.groupby(['Case'])['ec관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t4.drop(['level_1'], axis = 1, inplace = True)
  test_x['ec관측치'] = t4

  t5 = test_x.groupby(['Case'])['시간당분무량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t5.drop(['level_1'], axis = 1, inplace = True)
  test_x['시간당분무량'] = t5

  t6 = test_x.groupby(['Case'])['시간당백색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t6.drop(['level_1'], axis = 1, inplace = True)
  test_x['시간당백색광량'] = t6

  t7 = test_x.groupby(['Case'])['시간당적색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t7.drop(['level_1'], axis = 1, inplace = True)
  test_x['시간당적색광량'] = t7

  t8 = test_x.groupby(['Case'])['시간당청색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t8.drop(['level_1'], axis = 1, inplace = True)
  test_x['시간당청색광량'] = t8

  t9 = test_x.groupby(['Case'])['시간당총광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
  t9.drop(['level_1'], axis = 1, inplace = True)
  test_x['시간당총광량'] = t9

  test_new_x = test_new
  test_new_x = test_new_x.groupby(['DAT','Case']).sum().reset_index()

  test_new_x = test_new_x.sort_values(by=['Case','DAT'], axis=0).reset_index()
  test_new_x.drop(['index'], axis = 1, inplace=True)


  for i in range(len(test_new_x)):
    #온도
    test_new_x.iloc[i,12] = test_x.iloc[0 + (3*i),4]
    test_new_x.iloc[i,13] = test_x.iloc[1 + (3*i),4]
    test_new_x.iloc[i,14] = test_x.iloc[2 + (3*i),4]
    #습도
    test_new_x.iloc[i,15] = test_x.iloc[0 + (3*i),5]
    test_new_x.iloc[i,16] = test_x.iloc[1 + (3*i),5]
    test_new_x.iloc[i,17] = test_x.iloc[2 + (3*i),5]
    #co2
    test_new_x.iloc[i,18] = test_x.iloc[0 + (3*i),6]
    test_new_x.iloc[i,19] = test_x.iloc[1 + (3*i),6]
    test_new_x.iloc[i,20] = test_x.iloc[2 + (3*i),6]
    #ec
    test_new_x.iloc[i,21] = test_x.iloc[0 + (3*i),7]
    test_new_x.iloc[i,22] = test_x.iloc[1 + (3*i),7]
    test_new_x.iloc[i,23] = test_x.iloc[2 + (3*i),7]
    #분무
    test_new_x.iloc[i,24] = test_x.iloc[0 + (3*i),8]
    test_new_x.iloc[i,25] = test_x.iloc[1 + (3*i),8]
    test_new_x.iloc[i,26] = test_x.iloc[2 + (3*i),8]
    #백색
    test_new_x.iloc[i,27] = test_x.iloc[0 + (3*i),9]
    test_new_x.iloc[i,28] = test_x.iloc[1 + (3*i),9]
    test_new_x.iloc[i,29] = test_x.iloc[2 + (3*i),9]
    #적색
    test_new_x.iloc[i,30] = test_x.iloc[0 + (3*i),10]
    test_new_x.iloc[i,31] = test_x.iloc[1 + (3*i),10]
    test_new_x.iloc[i,32] = test_x.iloc[2 + (3*i),10]
    #청색
    test_new_x.iloc[i,33] = test_x.iloc[0 + (3*i),11]
    test_new_x.iloc[i,34] = test_x.iloc[1 + (3*i),11]
    test_new_x.iloc[i,35] = test_x.iloc[2 + (3*i),11]
    #총광
    test_new_x.iloc[i,36] = test_x.iloc[0 + (3*i),12]
    test_new_x.iloc[i,37] = test_x.iloc[1 + (3*i),12]
    test_new_x.iloc[i,38] = test_x.iloc[2 + (3*i),12]



  test_new_x.drop(['obs_time','시간대','내부온도관측치','내부습도관측치','co2관측치','ec관측치'], axis = 1, inplace = True)

  test_x = test_new_x

  test_x = test_x.drop(test_x.filter(regex='시간당').columns, axis=1)


  test_x['하루평균온도'] = (test_x['05시내부온도관측치누적'] + test_x['19시내부온도관측치누적'] + test_x['23시내부온도관측치누적']) / 3
  test_x['하루평균습도'] = (test_x['05시내부습도관측치누적'] + test_x['19시내부습도관측치누적'] + test_x['23시내부습도관측치누적']) / 3
  test_x['하루평균co2'] = (test_x['05시co2관측치누적'] + test_x['19시co2관측치누적'] + test_x['23시co2관측치누적']) / 3
  test_x['하루평균ec'] = (test_x['05시ec관측치누적'] + test_x['19시ec관측치누적'] + test_x['23시ec관측치누적']) / 3
  test_x['하루평균분무량'] = (test_x['05시분무량누적'] + test_x['19시분무량누적'] + test_x['23시분무량누적']) / 3
  test_x['하루평균백색광'] = (test_x['05시백색광누적'] + test_x['19시백색광누적'] + test_x['23시백색광누적']) / 3
  test_x['하루평균적색광'] = (test_x['05시적색광누적'] + test_x['19시적색광누적'] + test_x['23시적색광누적']) / 3
  test_x['하루평균청색광'] = (test_x['05시청색광누적'] + test_x['19시청색광누적'] + test_x['23시청색광누적']) / 3
  test_x['하루평균총광량'] = (test_x['05시총광량누적'] + test_x['19시총광량누적'] + test_x['23시총광량누적']) / 3

  test_x = test_x.drop(test_x.filter(regex = '총광').columns, axis =1)


  test_x['ec_x_분무05'] = (test_x['05시ec관측치누적']+1) * (test_x['05시분무량누적']+1)
  test_x['ec_x_분무19'] = (test_x['19시ec관측치누적']+1) * (test_x['19시분무량누적']+1)
  test_x['ec_x_분무23'] = (test_x['23시ec관측치누적']+1) * (test_x['23시분무량누적']+1)
  test_x['ec_x_분무평균'] = (test_x['하루평균ec']+1) * (test_x['하루평균분무량']+1)




  test_x['적색_+_청색05'] = (test_x['05시적색광누적']) + (test_x['05시청색광누적'])
  test_x['적색_+_청색19'] = (test_x['19시적색광누적']) + (test_x['19시청색광누적'])
  test_x['적색_+_청색23'] = (test_x['23시적색광누적']) + (test_x['23시청색광누적'])
  test_x['적색_+_청색평균'] = (test_x['하루평균적색광']) + (test_x['하루평균청색광'])


  test_x['수분량합'] =  test_x['측정될수분량1'] + test_x['측정될수분량2'] + test_x['측정될수분량3']

  test_x['수분량합12'] =  test_x['측정될수분량1'] + test_x['측정될수분량2']

  test_x['수분량합13'] =  test_x['측정될수분량1'] + test_x['측정될수분량3']

  test_x['수분량합23'] =  test_x['측정될수분량2'] + test_x['측정될수분량3']

  test_x.drop(test_x.filter(regex = '백색'), axis = 1, inplace=True)

  test_x.drop(test_x.filter(regex = '청색'), axis = 1, inplace=True)

  test_x.drop(['측정될수분량2','측정될수분량1','측정될수분량3'],axis=1,inplace=True)



  basic_col = test_x.columns
  kal_test = test_x

  for i in range(2,34):
    test_x['kf_X_'+str(i)]=0

  for j in range(0,len(test_x.Case.unique())): #수정 Data_Leakage
    #kal = test_x[test_x['Case'] == j]
    kal = test_x
    for i in (range(len(basic_col))):
        if((i == 0) | (i == 1)):
          continue
        current=0
        sum_c=[]
        z = kal.loc[:, kal.columns[i]]
        a = []           #필터링 된 피쳐(after)
        b = []           #필터링 전 피쳐(before)
        my_filter = KalmanFilter(dim_x=2,dim_z=1) #create kalman filter
        my_filter.x = np.array([[2.],[0.]])       # initial state (location and velocity)
        my_filter.F = np.array([[1.,1.], [0.,1.]])    # state transition matrix
        my_filter.H = np.array([[1.,0.]])    # Measurement function
        my_filter.P *= 1000.                 # covariance matrix
        my_filter.R = 5                      # state uncertainty
        my_filter.Q = Q_discrete_white_noise(dim = 2,dt=.1,var=.5) # process uncertainty   
        for k in z.values:
            my_filter.predict()
            my_filter.update(k)
            # do something with the output
            x = my_filter.x
            a.extend(x[0])
            b.append(k)
        sum_c=sum_c+a
        #test_x['kf_X_'+str(i)][test_x['Case'] == j] = sum_c
        test_x['kf_X_'+str(i)] = sum_c
  #fillna도 해줘야함

  copy_test_x = test_x.copy()

  copy_test_x.drop(copy_test_x.filter(regex = 'kf').columns, axis = 1, inplace = True)

  raw_cols_te = copy_test_x.columns
  new_test_x = pd.DataFrame()
  for i in range(0,len(copy_test_x.Case.unique())):
    #target = copy_test_x[copy_test_x['Case'] == i]
    target = copy_test_x
    mean_arr = []
    median_arr = []


    pbar = (target.columns[:])
    for column in pbar:
        column_list = target[column].to_list()
        for i in range(7):
          if((i) == (len(column_list))):
            break
          mean_arr.append(column_list[i])
          median_arr.append(column_list[i])

            
        for i in range(7, len(column_list)):
            mean_arr.append(float(np.mean(column_list[i-7:i])))
            median_arr.append(float(np.median(column_list[i-7:i])))
        target[f'{column}_mean_7'] = mean_arr
        target[f'{column}_median_7'] = median_arr
        mean_arr = []
        median_arr = []
    
    new_test_x = pd.concat([new_test_x, target], axis=0)
  test_x_moving_7 = new_test_x.drop(raw_cols_te, axis=1)

  test_x = pd.concat([test_x, test_x_moving_7], axis=1)

  raw_cols_te = copy_test_x.columns



  new_test_x = pd.DataFrame()

  for i in range(0,len(copy_test_x.Case.unique())):
    #target = copy_test_x[copy_test_x['Case'] == i]
    target = copy_test_x
    mean_arr = []
    median_arr = []


    pbar = (target.columns[:])
    for column in pbar:
        column_list = target[column].to_list()
        for i in range(14):
          if((i) == (len(column_list))):
            break
          mean_arr.append(column_list[i])
          median_arr.append(column_list[i])
            
        for i in range(14, len(column_list)):
            mean_arr.append(float(np.mean(column_list[i-14:i])))
            median_arr.append(float(np.median(column_list[i-14:i])))
        target[f'{column}_mean_14'] = mean_arr
        target[f'{column}_median_14'] = median_arr
        mean_arr = []
        median_arr = []
    
    new_test_x = pd.concat([new_test_x, target], axis=0)
  test_x_moving_14 = new_test_x.drop(raw_cols_te, axis=1)

  test_x = pd.concat([test_x, test_x_moving_14], axis=1)

  test_x_fil = test_x.iloc[:,1:34]


  new_test_x = pd.DataFrame()
  for i in range(0,len(copy_test_x.Case.unique())):
    #target = test_x_fil[test_x_fil['Case'] == i]
    target = test_x_fil
    test_merge = LPF(target,0.1,1)
    test_merge = pd.DataFrame(test_merge)
    
    new_test_x = pd.concat([new_test_x, test_merge], axis=0)
  new_test_x = new_test_x.add_suffix('_LPF')
  new_test_x = new_test_x.reset_index(drop = True)
  test_x = pd.concat([test_x,new_test_x], axis = 1)


  test_x.drop(['0_LPF'],axis = 1, inplace=True)
  test_x.drop(test_x.filter(regex = 'Case'),axis=1, inplace=True)
  if(day == 27):
    test_x.to_csv('last_day_fe.csv',index=False)
  return test_x

# 새 섹션

---
Final  
---


In [8]:
!pip install sdv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 7.5 MB/s 
     |████████████████████████████████| 1.6 MB 28.2 MB/s 
     |████████████████████████████████| 61 kB 408 kB/s 
     |████████████████████████████████| 139 kB 44.3 MB/s 
     |████████████████████████████████| 47 kB 5.1 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 9.4 MB 52.5 MB/s 
     |████████████████████████████████| 965 kB 62.3 MB/s 
     |████████████████████████████████| 295 kB 76.5 MB/s 
     |████████████████████████████████| 662 kB 63.4 MB/s 
     |████████████████████████████████| 15.3 MB 41.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
   

In [1]:
!pip install Autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 5.9 MB/s 
     |████████████████████████████████| 226 kB 23.5 MB/s 
     |████████████████████████████████| 62 kB 774 kB/s 
     |████████████████████████████████| 286 kB 61.9 MB/s 
     |████████████████████████████████| 103 kB 76.9 MB/s 
     |████████████████████████████████| 289 kB 89.2 MB/s 
     |████████████████████████████████| 49 kB 7.7 MB/s 
     |████████████████████████████████| 41 kB 341 kB/s 
     |████████████████████████████████| 802 kB 30.4 MB/s 
     |████████████████████████████████| 132 kB 80.8 MB/s 
     |████████████████████████████████| 1.0 MB 45.5 MB/s 
     |████████████████████████████████| 1.6 MB 49.4 MB/s 
     |████████████████████████████████| 60.2 MB 1.1 MB/s 
     |████████████████████████████████| 280 kB 54.4 MB/s 
     |████████████████████████████████| 58 kB 6.5 MB/s 
     |████████████████████████████████| 4

In [10]:
# CTGAN
from sdv.tabular import CTGAN
import pandas as pd
import warnings
import joblib
import os

In [ ]:
# CTGAN
from sdv.tabular import CTGAN
import pandas as pd
import warnings
import joblib
import os
#from gan_feature.gan_make_dataset import make_data
from autogluon.tabular import TabularDataset, TabularPredictor

warnings.filterwarnings('ignore')

input_path = "/content/drive/MyDrive/상추/original_data/train_input/*.csv"
target_path = "/content/drive/MyDrive/상추/original_data/train_target/*.csv"
generated_path = "/content/drive/MyDrive/상추/generate_data/"

input_path1 = "/content/drive/MyDrive/상추/original_data/train_input/CASE_13.csv"
input_path2 = "/content/drive/MyDrive/상추/original_data/train_input/CASE_14.csv"



class CreateGE:
    def __init__(self):
        self.gan_model = CTGAN()
        self.load_path = "/content/drive/MyDrive/상추/auto_models"
        self.raw_data = make_dataset(input_path, target_path)
        self.discrete_col = ['obs_time']
        self.predictor = TabularPredictor.load(self.load_path, require_version_match=False)

    def generated_data(self, input_path1, input_path2):
        print("Start generated_data")
        for i in range(28):
            print(f"=======Start Day{i}===========")
            raw_data1 = make_traindata_ctgan(input_path1)  # ctgan에 학습할 데이터 형태 바꿔줌.
            raw_day1 = raw_data1.iloc[i:i+1]  # day i 가져옴
            raw_data2 = make_traindata_ctgan(input_path2)  # ctgan에 학습할 데이터 형태 바꿔줌.
            raw_day2 = raw_data2.iloc[i:i+1]  # day i 가져옴
            self.gan_model.fit(pd.concat([raw_day1, raw_day2]))
            ctgan_data = self.gan_model.sample(100, randomize_samples=0)
            ctgan_data = make_raw(ctgan_data, i)
            ctgan_data.to_csv(os.path.join(
                generated_path, f'generate_day{i}.csv'), index=False)
            print(f"========End Day{i}=========\n\n")
        return None

    def growth_env(self):
        y_list = []
        for i in range(28):
            data = pd.read_csv(f"./generated_data/generate_day{i}.csv")  # 동적변경
            data = data.groupby(['obs_time']).mean().reset_index()
            preprocessed_data = make_data(data)
            y_prediction = self.predictor.predict(preprocessed_data)
            print(y_prediction)
            break
            # y_list.append(y_prediction)
            # max(y_list) # y_preddiction중 max값 반환해서 저장할 예정



### 선행

In [55]:
predict_model = TabularPredictor.load("/content/drive/MyDrive/상추/models_31_backup", require_version_match=False)
gen_data = pd.read_csv('/content/drive/MyDrive/상추/ctgan_day0_27/day0_100가지.csv')  # 동적변경
max_list = []
for i in range(10):
    gen_data_new = gen_data[i*24:(i+1)*24]
    gan_data = makedatasets(gen_data_new)
    y_pred = predict_model.predict(gan_data)
    max_list.append(y_pred.max())
    print(max_list)
max_list = pd.DataFrame(max_list)
print(max_list)
idx_max = max_list.idxmax()
print(f'idx_max: {idx_max}번째 index')
gen_dat = gen_data[int(idx_max.values)*24:(int(idx_max.values)+1)*24]
gen_dat['DAT'] = 0
gen_dat.to_csv("/content/day_00max.csv", index=False)

###################################################################################################################
    Case  DAT  obs_time    내부온도관측치    내부습도관측치      co2관측치     ec관측치  \
0    0.0  1.0         0  20.626915  57.146250  444.836735  2.682889   
1    0.0  1.0         1  20.274005  61.240953  442.235869  2.700584   
2    0.0  1.0         2  20.354893  61.298800  426.423077  2.559928   
3    0.0  1.0         3  20.616950  62.830509  426.920366  2.572541   
4    0.0  1.0         4  24.936538  51.571154  434.846154  2.537718   
5    0.0  1.0         5  24.969388  51.144406  433.302006  2.665307   
6    0.0  1.0         6  24.795918  50.705133  443.448980  2.553667   
7    0.0  1.0         7  25.288889  52.369400  437.962367  2.513436   
8    0.0  1.0         8  24.704451  53.950000  424.053571  2.518999   
9    0.0  1.0         9  24.983453  54.453260  417.186181  2.548894   
10   0.0  1.0        10  24.751923  53.168179  412.919693  2.597018   
11   0.0  1.0        11  24.8018

100%|██████████| 34/34 [00:00<00:00, 597.22it/s]

   DAT  Case  05시내부온도관측치누적  19시내부온도관측치누적  23시내부온도관측치누적  05시내부습도관측치누적  \
0  1.0   0.0    131.778689    490.392042    592.222683    345.232071   

   19시내부습도관측치누적  23시내부습도관측치누적  05시co2관측치누적  19시co2관측치누적  ...      하루평균분무량  \
0   1095.573502   1278.555259  2608.564205   8438.94335  ...  14032.06662   

        하루평균적색광     ec_x_분무05      ec_x_분무19     ec_x_분무23      ec_x_분무평균  \
0  87546.675885  70477.630947  939161.558022  1.283755e+06  623935.122514   

         수분량합     수분량합12     수분량합13      수분량합23  
0  270.470359  137.46925  200.42272  203.048749  

[1 rows x 34 columns]



100%|██████████| 34/34 [00:00<00:00, 1272.24it/s]


[3.8618743]
###################################################################################################################
    Case  DAT  obs_time    내부온도관측치    내부습도관측치      co2관측치     ec관측치  \
24   1.0  1.0         0  20.822985  57.146250  444.836735  2.682889   
25   1.0  1.0         1  20.280769  61.240953  447.557692  2.552472   
26   1.0  1.0         2  20.359615  61.472827  449.901961  2.559928   
27   1.0  1.0         3  20.616950  62.830509  430.420399  2.551349   
28   1.0  1.0         4  24.936538  51.485859  434.846154  2.537718   
29   1.0  1.0         5  24.969388  49.164490  437.960649  2.570052   
30   1.0  1.0         6  24.795918  51.931428  443.448980  2.661637   
31   1.0  1.0         7  25.090091  50.387405  440.074074  2.513436   
32   1.0  1.0         8  24.898182  53.950000  424.053571  2.518999   
33   1.0  1.0         9  24.878362  54.453260  416.122449  2.677423   
34   1.0  1.0        10  25.098182  53.676923  416.490909  2.686095   
35   1.0  1.0       

...

[3.8618743, 3.7397292, 3.7793343, 3.7373142, 3.688678, 3.727151, 3.7023962, 3.7564979, 3.7180326]
###################################################################################################################
     Case  DAT  obs_time    내부온도관측치    내부습도관측치      co2관측치     ec관측치  \
216   9.0  1.0         0  20.475510  64.119231  444.836735  2.682889   
217   9.0  1.0         1  20.220815  65.353123  447.557692  2.570820   
218   9.0  1.0         2  20.178691  62.007009  433.498324  2.674136   
219   9.0  1.0         3  20.616950  57.910060  421.745763  2.551349   
220   9.0  1.0         4  24.936538  51.571154  434.846154  2.634618   
221   9.0  1.0         5  24.940816  51.240213  437.318147  2.554094   
222   9.0  1.0         6  24.795918  51.931428  443.448980  2.564102   
223   9.0  1.0         7  25.288889  49.603778  436.030909  2.635410   
224   9.0  1.0         8  24.898182  53.950000  426.194446  2.518999   
225   9.0  1.0         9  25.008696  52.567233  416.122449  2.5208

100%|██████████| 34/34 [00:00<00:00, 767.91it/s]

   DAT  Case  05시내부온도관측치누적  19시내부온도관측치누적  23시내부온도관측치누적  05시내부습도관측치누적  \
0  1.0   9.0    131.369321    489.988971     591.92562    352.200789   

   19시내부습도관측치누적  23시내부습도관측치누적  05시co2관측치누적  19시co2관측치누적  ...       하루평균분무량  \
0   1099.735655   1284.124312  2619.802814  8446.799574  ...  14259.946263   

        하루평균적색광     ec_x_분무05      ec_x_분무19     ec_x_분무23      ec_x_분무평균  \
0  90427.777895  67349.481221  954274.119888  1.290858e+06  627211.640288   

         수분량합      수분량합12      수분량합13      수분량합23  
0  267.601711  137.250716  197.728501  200.224206  

[1 rows x 34 columns]



100%|██████████| 34/34 [00:00<00:00, 1446.05it/s]


[3.8618743, 3.7397292, 3.7793343, 3.7373142, 3.688678, 3.727151, 3.7023962, 3.7564979, 3.7180326, 3.7328546]
          0
0  3.861874
1  3.739729
2  3.779334
3  3.737314
4  3.688678
5  3.727151
6  3.702396
7  3.756498
8  3.718033
9  3.732855
idx_max: 0    0
dtype: int64번째 index


In [56]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd


warnings.filterwarnings('ignore')

predict_model = TabularPredictor.load("/content/drive/MyDrive/상추/models_31_backup", require_version_match=False)

for day in range(1, 28):
    max_0 = pd.read_csv('/content/day_00max.csv')
    print(f'{day}일차 Start')
    gen_data = pd.read_csv(f"/content/drive/MyDrive/상추/ctgan_day0_27/day{day}_100가지.csv")  # 동적변경
    max_list = []
    for i in range(10):
        gen_data_new = gen_data[i*24:(i+1)*24]
        gen_data_new = gen_data_new.reset_index(drop=True)
        max_0['Case'] = i
#이거를 max_0 말고 gen_data_new에 처리해주면 나중에 concat해줄 때 +값이 처리가 안된게 들어가서 그 다음날에 적용이 안되어있음
        gen_data_new['Case'] = i
        gen_data_new['DAT'] = day 
        gan_data = make_data(gen_data_new, max_0, day)
        y_pred = predict_model.predict(gan_data)
        '''
        if((day== 3) &(i == 0)):
          gan_data.to_csv('/content/check.csv',index=False)'''
        max_list.append(y_pred.max())
        print(max_list)
    max_list = pd.DataFrame(max_list)
    print(max_list)
    idx_max = max_list.idxmax()
    print('idx_max: %d번째 index' %(idx_max))
    gen_dat = gen_data[int(idx_max.values)*24:(int(idx_max.values)+1)*24]
    gen_dat['DAT'] = day
    con_dat = pd.concat([max_0, gen_dat], axis=0)
    con_dat.to_csv("/content/day_00max.csv", index=False)
    print(f'{day}일차 End')


1일차 Start
[5.899909]
[5.899909, 5.903298]
[5.899909, 5.903298, 5.956066]
[5.899909, 5.903298, 5.956066, 5.9633203]
[5.899909, 5.903298, 5.956066, 5.9633203, 5.958065]
[5.899909, 5.903298, 5.956066, 5.9633203, 5.958065, 5.890661]
[5.899909, 5.903298, 5.956066, 5.9633203, 5.958065, 5.890661, 5.9838076]
[5.899909, 5.903298, 5.956066, 5.9633203, 5.958065, 5.890661, 5.9838076, 5.9346557]
[5.899909, 5.903298, 5.956066, 5.9633203, 5.958065, 5.890661, 5.9838076, 5.9346557, 5.984939]
[5.899909, 5.903298, 5.956066, 5.9633203, 5.958065, 5.890661, 5.9838076, 5.9346557, 5.984939, 5.887841]
          0
0  5.899909
1  5.903298
2  5.956066
3  5.963320
4  5.958065
5  5.890661
6  5.983808
7  5.934656
8  5.984939
9  5.887841
idx_max: 8번째 index
1일차 End
2일차 Start
[7.6042585]
[7.6042585, 7.6060324]
[7.6042585, 7.6060324, 7.652953]
[7.6042585, 7.6060324, 7.652953, 7.60509]
[7.6042585, 7.6060324, 7.652953, 7.60509, 7.605056]
[7.6042585, 7.6060324, 7.652953, 7.60509, 7.605056, 7.570759]
[7.6042585, 7.6060324, 

In [ ]:
# gan_model = CTGAN()
# input_path1 = "./data/train_input/CASE_13.csv"
# input_path2 = "./data/train_input/CASE_14.csv"
# generated_path = "./generate_data/"
# print("Start generated_data")
# for i in range(28):
#     print(f"=======Start Day{i}===========")
#     raw_data1 = make_traindata_ctgan(input_path1)  # ctgan에 학습할 데이터 형태 바꿔줌.
#     raw_day1 = raw_data1.iloc[i:i+1]  # day i 가져옴
#     raw_data2 = make_traindata_ctgan(input_path2)  # ctgan에 학습할 데이터 형태 바꿔줌.
#     raw_day2 = raw_data2.iloc[i:i+1]  # day i 가져옴
#     gan_model.fit(pd.concat([raw_day1, raw_day2]))
#     ctgan_data = gan_model.sample(100, randomize_samples=0)
#     ctgan_data = make_raw(ctgan_data, i)
#     ctgan_data.to_csv(os.path.join(
#         generated_path, f'generate_day{i}.csv'), index=False)
#     print(f"========End Day{i}=========\n\n")
